In [0]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import pickle
from math import sqrt
import matplotlib.pyplot as plt
import random
from keras import backend as K
from keras.preprocessing.image import img_to_array,array_to_img
from scipy.ndimage import affine_transform
import keras
from PIL import Image
from keras import applications,optimizers
from keras.models import Model,Sequential,Input
from keras.layers import Dense,Activation,Lambda,Concatenate,Reshape
from keras.utils import Sequence
from tqdm import tqdm_notebook as tqdm
import time 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
gdrive	sample_data


In [0]:
%cd gdrive/
!ls

/content/gdrive
'My Drive'


In [0]:
%cd 'My Drive'
!ls

/content/gdrive/My Drive
 Assessment.gform
 Attendance.gsheet
'AW.Programming.Principles.and.Practice.Using.Cplusplus.2nd.Edition.0321992784 (1).pdf'
 AW.Programming.Principles.and.Practice.Using.Cplusplus.2nd.Edition.0321992784.pdf
'Colab Notebooks'
'Control Engineering '
 Control.pdf
 CSL100
'CV Format2.pdf'
'Digital Logic'
'Digital Project.gsheet'
'Electromagnetics '
 gpu_drive_help.ipynb
 Humpback-whale-identification
'IITJ-1702_Calendar_Time Table_Attendance Policy.pdf'
'Image Quality'
 IMG-20190531-WA0001.jpg
 IMG-20190612-WA0000.jpg
'_InternshalaResume (1).pdf'
 _InternshalaResume.pdf
 kaggle.json
 Kuo-NetworkAnalysisSynthesis_text.pdf
'lab3 control.zip'
'LAB MANUAL-IITJ 2019 (1).gdoc'
'LAB MANUAL-IITJ 2019.gdoc'
'Low rate Energy'
'[Morris,_David_Morris]_Stereochemistry_(Tutorial_C(b-ok.org).pdf'
 Mumbai
'Network Intrusion Detection'
 NIDS.odp
'Oppenheim - Signals And Systems (Complete).pdf'
'Parking detection'
'[P._C._Sen]_Principles_of_Electric_Machines_and_Po(b-ok.org).pdf'
 

In [0]:
%cd  Humpback-whale-identification

/content/gdrive/My Drive/Humpback-whale-identification


In [0]:
np.random.seed(42)
TRAIN_CSV='train.csv'
TRAIN_DIR='train/'
TEST_DIR='test/'
BB='bounding_boxes.csv'
P2H='p2h.pickle'
P2S='p2size.pickle'
tagged=dict([(p,w) for a,p,w in pd.read_csv(TRAIN_CSV).to_records()])
print(tagged)

{'0000e88ab.jpg': 'w_f48451c', '0001f9222.jpg': 'w_c3d896a', '00029d126.jpg': 'w_20df2c5', '00050a15a.jpg': 'new_whale', '0005c1ef8.jpg': 'new_whale', '0006e997e.jpg': 'new_whale', '000a6daec.jpg': 'w_dd88965', '000f0f2bf.jpg': 'new_whale', '0016b897a.jpg': 'w_64404ac', '001c1ac5f.jpg': 'w_a6f9d33', '001cae55b.jpg': 'w_581ba42', '001d7450c.jpg': 'new_whale', '00200e115.jpg': 'new_whale', '00245a598.jpg': 'new_whale', '002b4615d.jpg': 'new_whale', '002f99f01.jpg': 'new_whale', '00355ff28.jpg': 'w_cb622a2', '00357e37a.jpg': 'w_d3b46e7', '003795857.jpg': 'new_whale', '0041880bf.jpg': 'new_whale', '0043da555.jpg': 'new_whale', '00442c882.jpg': 'w_8cad422', '00464ff65.jpg': 'new_whale', '004775679.jpg': 'w_13ae3d4', '004ae9e26.jpg': 'new_whale', '004c0f43b.jpg': 'new_whale', '004e8ad5b.jpg': 'w_3de579a', '004f87702.jpg': 'w_1d0830e', '0050ef29d.jpg': 'w_2ba6978', '00514c876.jpg': 'w_8dddbee', '0052ce2f5.jpg': 'w_2365d55', '00537ec91.jpg': 'w_3879fa0', '00570db6b.jpg': 'w_9c506f6', '005ce310

In [0]:
bounding_pd=pd.read_csv(BB)   #Bounding Box Coordinates
bounding_pd.head()
boxes_coord=dict( [(p,(x0,y0,x1,y1)) for a,p,x0,y0,x1,y1 in pd.read_csv(BB).to_records()])
print(boxes_coord)

{'72c3ce75c.jpg': (0, 0, 1045, 389), 'a7ad640ee.jpg': (35, 71, 523, 291), 'df2b6c364.jpg': (12, 15, 1033, 308), '26013fcb5.jpg': (3, 1, 1024, 292), '09eff7b37.jpg': (11, 8, 926, 334), '321f922d3.jpg': (23, 87, 600, 210), 'a08161913.jpg': (3, 24, 547, 274), 'd2dde23b0.jpg': (18, 219, 1029, 461), '111d1ab30.jpg': (31, 48, 1042, 239), 'cec27e4e3.jpg': (58, 185, 962, 487), '89acede28.jpg': (56, 131, 996, 313), 'b592e25d4.jpg': (41, 73, 1005, 323), '621c3ed03.jpg': (0, 293, 1028, 535), '86517d536.jpg': (24, 55, 438, 205), 'c5f4b73be.jpg': (13, 100, 982, 507), '5afaf2ffd.jpg': (168, 76, 789, 315), 'b060f5529.jpg': (6, 62, 988, 414), '7919aa309.jpg': (99, 32, 938, 592), '5a35f6ded.jpg': (61, 61, 1016, 508), 'f25a1c3f5.jpg': (21, 235, 986, 513), '8d4453e81.jpg': (9, 304, 1017, 572), '3b2468366.jpg': (12, 95, 878, 334), '929b9b616.jpg': (22, 123, 1012, 345), 'e2deae874.jpg': (121, 129, 878, 514), '6802020c8.jpg': (51, 0, 991, 581), '1b5501c54.jpg': (23, 77, 725, 240), '54cfc2e29.jpg': (12, 37, 

In [0]:
h2ws={}
for k,v in tagged.items():
    if v!='new_whale':
        if v not in h2ws: h2ws[v]=[k]
        else: h2ws[v].append(k)
            
print(len(h2ws))   

train=[]
for k,v in h2ws.items():
    if len(v)>1:
        train+=v
random.shuffle(train)        
print(len(train))        

w2ts={}

for k,v in tagged.items():
    if k in train:
        if v not in w2ts: w2ts[v]=[k]
        else: w2ts[v].append(k)
            
for k,v in w2ts.items():
    w2ts[k]=np.array(v)
    
print(len(w2ts)) 


5004
13624
2931


In [0]:
position={}

for i,t in enumerate(train):
    position[t]=i
    
print(position)    
pickle_out = open("pos.pickle","wb")
pickle.dump(position, pickle_out)
pickle_out.close()

{'b2db5d2f6.jpg': 0, '53bb428e9.jpg': 1, 'd9f49579b.jpg': 2, 'edc8b404f.jpg': 3, 'c679073c0.jpg': 4, '75385c608.jpg': 5, '90380ed4c.jpg': 6, 'bd22cbbb2.jpg': 7, '295324981.jpg': 8, '0cfed7024.jpg': 9, '05cb42d60.jpg': 10, 'c5f4b73be.jpg': 11, '1c565ff94.jpg': 12, '37b04f82e.jpg': 13, 'b758f5366.jpg': 14, '2f4c7603e.jpg': 15, '08406b193.jpg': 16, '39a3458a3.jpg': 17, '0db1edb0c.jpg': 18, 'd5098b5a1.jpg': 19, '32e51c323.jpg': 20, '0adaaefda.jpg': 21, 'ee108f1cd.jpg': 22, '425f763ca.jpg': 23, '69f294f9c.jpg': 24, 'a625592c3.jpg': 25, '0c341777d.jpg': 26, 'ec038eee7.jpg': 27, '35422a88d.jpg': 28, 'fd46b46d0.jpg': 29, '6f8c84ab6.jpg': 30, 'edb93f9d0.jpg': 31, 'bdab6a183.jpg': 32, 'adf2052e7.jpg': 33, '303938df6.jpg': 34, 'e217cb745.jpg': 35, '03bb5b068.jpg': 36, '21af3144a.jpg': 37, 'a3c12257e.jpg': 38, '15c85e660.jpg': 39, '9c09ecb6d.jpg': 40, '047ca0d72.jpg': 41, '061996245.jpg': 42, '3a96fc020.jpg': 43, 'bb7b4491b.jpg': 44, '2f0b9fe47.jpg': 45, '331051b10.jpg': 46, '031e2f2e0.jpg': 47, '

In [0]:
from os.path import isfile
def expand_path(file):
    if isfile(TRAIN_DIR+file): return TRAIN_DIR+file
    elif isfile(TEST_DIR+file): return TEST_DIR+file

In [0]:
img_shape=(299,299,3)
img_shape2=(299,299,1)
anisotropy=2.15
crop_margin=0.05

def read_raw_image(p):
    img = Image.open(expand_path(p))
    return img

def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    """
    Build a transformation matrix with the specified characteristics.
    """
    rotation        = np.deg2rad(rotation)
    shear           = np.deg2rad(shear)
    rotation_matrix = np.array([[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix    = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix     = np.array([[1.0/height_zoom, 0, 0], [0, 1.0/width_zoom, 0], [0, 0, 1]])
    shift_matrix    = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))

def read_cropped_image(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
   
    img   = read_raw_image(p).convert('L')
    img   = img_to_array(img)
    
    size_x,size_y,z = img.shape
    
    # Determine the region of the original image we want to capture based on the bounding box.
    x0,y0,x1,y1   = boxes_coord[p]
    
    dx            = x1 - x0
    dy            = y1 - y0
    x0           -= dx*crop_margin
    x1           += dx*crop_margin + 1
    y0           -= dy*crop_margin
    y1           += dy*crop_margin + 1
    if (x0 < 0     ): x0 = 0
    if (x1 > size_x): x1 = size_x
    if (y0 < 0     ): y0 = 0
    if (y1 > size_y): y1 = size_y
    dx            = x1 - x0
    dy            = y1 - y0
    if dx > dy*anisotropy:
        dy  = 0.5*(dx/anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx  = 0.5*(dy*anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5*img_shape[0]], [0, 1, -0.5*img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0)/img_shape[0], 0, 0], [0, (x1 - x0)/img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
        trans = np.dot(build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.8, 1.0),
            random.uniform(0.8, 1.0),
            random.uniform(-0.05*(y1 - y0), 0.05*(y1 - y0)),
            random.uniform(-0.05*(x1 - x0), 0.05*(x1 - x0))
            ), trans)
    trans = np.dot(np.array([[1, 0, 0.5*(y1 + y0)], [0, 1, 0.5*(x1 + x0)], [0, 0, 1]]), trans)

    
    
    # Apply affine transformation
    matrix = trans[:2,:2]
    offset = trans[:2,2]
    img    = img.reshape(img.shape[:-1])
    img    = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant', cval=np.average(img))
    img    = img.reshape(img_shape2)

    # Normalize to zero mean and unit variance
    img  -= np.mean(img, keepdims=True)
    img  /= np.std(img, keepdims=True) + K.epsilon()
    img=array_to_img(img)
    
    img=img.convert('RGB')
    
    img=img_to_array(img)
    
    return img

def read_for_training(p):
    
    return read_cropped_image(p, True)

def read_for_validation(p):
    return read_cropped_image(p,False)


In [0]:
###  BRANCH MODEL  ###
branch_model=applications.xception.Xception(include_top=True,weights='imagenet')
for layer in branch_model.layers[:]:
    layer.trainable=False
branch_model.summary()    

W0627 08:24:00.913993 140144619841408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0627 08:24:00.969099 140144619841408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0627 08:24:00.979605 140144619841408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0627 08:24:01.031284 140144619841408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0627 08:24:01.033054 1401446198

91889664/91884032 [==============================] - 8s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________

In [0]:
###   HEAD MODEL   ###
    
xa_inp = Input(shape=branch_model.output_shape[1:])
xb_inp = Input(shape=branch_model.output_shape[1:])
x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
x4 = Lambda(lambda x: K.square(x))(x3)
x5=  Lambda(lambda x: x[0]*K.log(x[1]))([xa_inp, xb_inp])
x6=  Lambda(lambda x: x[1]*K.log(x[0]))([xa_inp, xb_inp])
x = Concatenate()([x1, x2, x3, x4, x5 ,x6 ])
x = Reshape((6, branch_model.output_shape[1], 1), name='reshape1')(x)
 
    
    
x=Dense(6,input_dim=(),activation='relu')(x)
x=Dense(8,activation='relu')(x)
x=Dense(16,activation='relu')(x)
x=Dense(1,activation='softmax')(x)
#x= Lambda(lambda y: y)(x)

print(type(x))  

head_model = Model([xa_inp, xb_inp], x, name='head')

## Siamese Network ###
    
img_a=Input(shape=img_shape)
img_b=Input(shape=img_shape)
print(img_a.shape)
xa=branch_model(img_a)
xb=branch_model(img_b)
x=head_model([xa,xb])
model_final=Model([img_a,img_b],x)
model_final.compile(loss='mean_squared_error',optimizer=optimizers.Adam(0.001),metrics=['acc'])
    

<class 'tensorflow.python.framework.ops.Tensor'>
(?, 299, 299, 3)


W0627 08:24:29.033741 140144619841408 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
features=[]
for i in tqdm(range(len(train))):
    temp=np.zeros((1,299,299,3))
    temp[0]=read_for_validation(train[i])/255
    p=branch_model.predict(temp)
    
    features.append(p)
 



In [0]:
pickle_out = open("features.pickle","wb")
pickle.dump(features, pickle_out)
pickle_out.close()

In [0]:
import gc
score=np.zeros((len(train),len(train)))
for j in tqdm(range(len(train))):
    for i in range(len(train)):
        score[j][i]=np.sum(np.abs(features[i]-features[j]))
        
   

In [0]:
score=np.round(score*1000)
score=-score
score

array([[   -0., -1249., -1295., ...,  -903.,  -933.,  -919.],
       [-1249.,    -0., -1609., ..., -1501., -1189., -1549.],
       [-1295., -1609.,    -0., ...,  -447., -1421.,  -415.],
       ...,
       [ -903., -1501.,  -447., ...,    -0., -1223.,  -213.],
       [ -933., -1189., -1421., ..., -1223.,    -0., -1290.],
       [ -919., -1549.,  -415., ...,  -213., -1290.,    -0.]])

In [0]:
import pickle
pickle_out = open("dict.pickle","wb")
pickle.dump(score, pickle_out)
pickle_out.close()

In [0]:
!pip3 install lapjv

from lapjv import lapjv

  Stored in directory: /root/.cache/pip/wheels/6e/fe/fd/b700bd67e00bcd65241d55d875eec17892a86ca6a4deef9515
Successfully built lapjv


In [0]:
pickle_in=open('dict.pickle','rb')
score=pickle.load(pickle_in)
pickle_in=open('pos.pickle','rb')
position=pickle.load(pickle_in)

In [0]:
np.min(score)

-1999.0

In [0]:
max_len=0
for v,k in h2ws.items():
    if len(k)>1:
        if len(k)>max_len:
            max_len=len(k)
        for i in k:
            for j in k:
                score[position[i]][position[j]]=0

In [0]:
reverse_position={}
for k,v in position.items():
    reverse_position[v]=k

In [0]:
score=2000+score
np.max(score)

2000.0

In [0]:
np.min(score)

1.0

In [0]:
pairs={}
for i in tqdm(range(max_len)):
    
    row,col,a=lapjv(score)
    
    for i in range(len(row)):
        if reverse_position[row[i]] not in pairs.keys():
            pairs[reverse_position[row[i]]]=[reverse_position[col[i]]]
        else:
            pairs[reverse_position[row[i]]].append(reverse_position[col[i]])
        
    
        score[row[i]][col[i]]=2000
        
     
        
        
    

In [0]:
import pickle
pickle_out = open("pair.pickle","wb")
pickle.dump(pairs, pickle_out)
pickle_out.close()

In [0]:
pickle_out=open('pair.pickle','rb')
pairs=pickle.load(pickle_out)

In [0]:
training_data=[]
y=[]
for k,v in h2ws.items():
  temp_len=len(v)
  if temp_len>1:
    for i in v:
      for j in v:
        training_data.append((i,j))
        y.append(1)
      
    for i in v:
      for j in range(temp_len):
        training_data.append((i,pairs[i][j]))
        y.append(0)
      
      
  

In [0]:
X_train=[]
for i in traning_data:
  x,y=i
  x=read_for_validation(x)
  y=read_for_validation(y)
  
  X_train.append([x,y])

In [0]:
model_final.fit(X_train,y,epochs=32.verbose=1,batch_size=32)